In [1]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd
import time

In [2]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}


In [3]:
def get_links(year, month:str, page):
    url ='http://www.arthub.co.kr/sub01/board033_list.htm?'
    seed = 'http://www.arthub.co.kr/sub01/'
    
    params = {
        'k_date':year,
        'k_month':month,
        'PHPSESSID':'e124fc854822909d9ec3618c1e6089f2',
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('a.sub01_list_title')
    
    return [(str(year)+month,a.get_text(),seed+a['href']) for a in aList]

In [37]:
get_links(2018,'09',1)

[('201809',
  '이재건展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=25924'),
 ('201809',
  'An unfamiliar gaze - 정의철展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26876'),
 ('201809',
  '조성희展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=25935'),
 ('201809',
  '서근범展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26893')]

html 태그가 매우 드럽게 달려있다

부적절한 tree 구조인데다가 identification 규칙이 일정치 않음, 일일이 찾기힘듦

In [277]:
artists = [t3.text for t3 in t3.find('td',{'width':'339'}).find_all('a')]
date = [el for el in t3.find('span',{'class':'red_point'}).parent]
time = [el.text for el in t3.find_all('span',{'class':'red_point'})[1:]]

전시정보를 가져오는 함수 get_info를 짰는데 next.next.next... 이런식으로 드럽다, 어쩔수없었다

In [4]:
def get_info(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    t3 = soup.select('table')[3]

    info = []
    for el in [el.next.next.next.next.next for el in t3.find_all('td',{'width':'84','height':'18','align':'left'})]: #전시정보 뽑아오기, tag가 드러움
        if el.string==None: #class 'bs4.element.Tag'
            info.append(el.text)
        else:                          #class 'bs4.element.NavigableString'
            info.append(el.string) 

    artists = list(map(lambda s:s.strip(), info[0].split(',')))
    articles = [td.next['href'] for td in t3.find_all('td',{'width':'510'})[1:]]

    del info[0]
    info.insert(0,artists)
    info.append(articles)
    
    return info

In [231]:
get_info('http://www.arthub.co.kr/sub01/board03_view.htm?No=26567')

[['강지만',
  '김경섭',
  '김용석',
  '김아영',
  '문기전',
  '박정원',
  '박정용',
  '박흥선',
  '성태진',
  '송지연',
  '윤기원',
  '이수정',
  '이재열',
  '진선미'],
 '2018-08-01 ▶ 2018-08-04',
 '없음',
 'Open 14:00 ~ Close 20:00',
 '아트팩토리 후(ART FACTORY hoo)\xa0 다른전시 보기',
 '강원도 원주시 문막읍 비야동길 10-12 (후용리 1318) 나동창고 ',
 '010-4014-7488',
 'www.facebook.com/artfactoryhoo',
 ['http://www.newsis.com/view/?id=NISX20180710_0000359747&cid=10701']]

텍스트를 가져오는 함수 get_text

In [5]:
def get_text(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    ps = [p.text for p in soup.find_all('p',{'align':'justify'})]
    text = ''.join(ps)
    
    return text.strip()

In [238]:
get_text('http://www.arthub.co.kr/sub01/board03_view.htm?No=19282')

'The birth of Life\n        물은 모든 생명의 원천이다. 모든 사물은 물에서 시작되어 그 끝 역시 물로 마무리 되고 그 과정은 지속적인 순환과정을 겪는다. \n        그리고 물의 순환과정을 통해 우리는 생명의 강인함과 진리를 깨닫게 된다. 또한 물에서 비롯된 생명은 하나가 모여 전체가 되듯이 \n        한 화면 속에서 개개의 소재들이 모여 하나의 화면을 구성하며 전일적인 개체로 성장한다.\n\n\n        The song of Life\n        모든 생명은 아름답고 작가의 작업은 그 아름다움을 노래하기 위한 수단이다. 그리고 그 아름다움을 만들어내는 모든 생명의 기원은 \n        물이다. 그래서 그의 작품 속에는 물을 상징하는 소재들이 그려져 있다. 또한 물위에서 기원하듯 그 주위에는 생명력을 품은 아름다운 \n        꽃들이 둘러싸며 저마다의 생명력을 뿜어내고 있다.The \n        energy of Life\n        생명은 항상 강인함을 지닌다. 어떠한 시련에도 생명의 순환은 결코 멈추지 않는다. 그리고 이것은 생명의 원천인 물이 가지는 에너지이기도 \n        하다. 작가는 이러한 끊임없는 순환의 과정 속의 생명이 가지는 강인함과 아름다움을 노래하기 위해 캔버스 위를 유화와 아크릴을 \n        통해 다채로운 색채로 표현한다. 잔잔한 한 폭의 정물화 인듯하지만 화려하고 과감한 색채의 대비는 우리에게 생명의 에너지를 속삭인다. \n         ⓒ'

샘플 데이터를 만들어보자, 55개의 전시만 가져왔다

In [1]:
start = time.time()

arts = []
for y in range(2008,2014):
    for m in tqdm(range(1,13)):
        if m<10:
            m = '0'+str(m)
        else:
            m = str(m)
        m_links = []
        for p in range(1,6): 
            try:
                for _ in range(2): #TimeoutError 방지위해 두번까지 시도한다
                    m_links.extend(get_links(y,m,p))
                    break
            except Exception as e:
                print(e)
                time.sleep(1)
                continue
                
        m_links = list(set(m_links)) #중복 링크 제거
        for link in m_links:
            try:
                info = get_info(link[2])
                text = get_text(link[2])

                temp = [*link,*info,text]
                arts.append(temp)
                
            except Exception as e:
                print(e)
                continue
                
end = time.time()
data = pd.DataFrame(arts,columns=['연월','전시제목','링크','전시작가','전시일정','초대일시','관람시간','전시장소','주소','연락처','홈페이지','아티클','텍스트'])
data.sample(5)

print(end-start)

In [2]:
len(data.전시제목.unique())

In [51]:
data[data.duplicated(subset='전시제목')]

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18309 entries, 0 to 18308
Data columns (total 13 columns):
연월      18309 non-null object
전시제목    18309 non-null object
링크      18309 non-null object
전시작가    18309 non-null object
전시일정    18309 non-null object
초대일시    18309 non-null object
관람시간    18308 non-null object
전시장소    18308 non-null object
주소      18308 non-null object
연락처     18308 non-null object
홈페이지    18308 non-null object
아티클     18292 non-null object
텍스트     18292 non-null object
dtypes: object(13)
memory usage: 1.8+ MB


In [8]:
data.to_excel('test2.xlsx',sheet_name='sheet1')

In [10]:
d = pd.read_excel('test2.xlsx',sheet_name='sheet1')

In [11]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18309 entries, 0 to 18308
Data columns (total 13 columns):
연월      18309 non-null int64
전시제목    18309 non-null object
링크      18309 non-null object
전시작가    18309 non-null object
전시일정    18309 non-null object
초대일시    11342 non-null object
관람시간    18308 non-null object
전시장소    18308 non-null object
주소      18292 non-null object
연락처     18308 non-null object
홈페이지    14182 non-null object
아티클     18292 non-null object
텍스트     17918 non-null object
dtypes: int64(1), object(12)
memory usage: 2.0+ MB


In [107]:
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8035 entries, 0 to 8034
Data columns (total 13 columns):
연월      8035 non-null int64
전시제목    8035 non-null object
링크      8035 non-null object
전시작가    8035 non-null object
전시일정    8035 non-null object
초대일시    7629 non-null object
관람시간    8034 non-null object
전시장소    8034 non-null object
주소      8034 non-null object
연락처     8034 non-null object
홈페이지    7432 non-null object
아티클     8034 non-null object
텍스트     7703 non-null object
dtypes: int64(1), object(12)
memory usage: 878.8+ KB


In [109]:
corpus = ''
for txt in data.텍스트:
    if txt!=None:
        corpus +=txt

In [110]:
# corpus #텍스트를 합쳐 코퍼스를 만든다
len(corpus)

14891082

아래부터는 텍스트 전처리 과정

In [111]:
import nltk
from konlpy.tag import Kkma,Twitter
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
from matplotlib import font_manager, rc

In [112]:
kkma = Kkma()
# twitter = Twitter()

In [3]:
# tokens = kkma.morphs(corpus)

In [ ]:
word_tokenize(corpus)[:15]

In [ ]:
count2 = Counter(tokens)
count2.most_common(30) #불용어 처리를 하지않은 결과이다

In [ ]:
text = nltk.Text(tokens)

In [ ]:
text.similar("작가")

In [ ]:
text.similar("존재")

In [ ]:
text.similar("희망")